<a href="https://colab.research.google.com/github/JASCRAJ/Sketch-literature/blob/main/200424.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install cairosvg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.1/75.1 kB 5.2 MB/s eta 0:00:00


In [ ]:
!pip install gradio_client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.1 MB/s eta 0:00:00


MIT Line drawing

In [ ]:
import os
from PIL import Image
import requests
from io import BytesIO
from gradio_client import Client

# Initialize Gradio client
client = Client("https://carolineec-informativedrawings.hf.space/")

# Define input and output directories
input_dir = "/content/drive/MyDrive/wikiart/Guided-by-Vector-Flow_input/"
output_dir = "/content/drive/MyDrive/wikiart/Guided-by-Vector-Flow_line"
os.makedirs(output_dir, exist_ok=True)

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# List all image files in the input directory
image_files = os.listdir(input_dir)

# Iterate through each image file
for image_file in image_files:
    # Construct the full path to the input image
    input_image_path = os.path.join(input_dir, image_file)

    # Make prediction
    result = client.predict(
        input_image_path,  # Image file path
        "style 1",         # Version
        api_name="/predict"
    )

    # Assuming the result contains the path to the output image
    output_img_path = result

    # Open the image from the path
    image = Image.open(output_img_path)

    # Save the image to the output directory with the same name
    output_image_path = os.path.join(output_dir, image_file)
    image.save(output_image_path)

    print(f"Processed image {image_file} and saved as {output_image_path}")

print("All images processed and saved successfully.")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loaded as API: https://carolineec-informativedrawings.hf.space/ ✔
Processed image hb.png and saved as /content/drive/MyDrive/wikiart/Guided-by-Vector-Flow_line/hb.png
Processed image h.jpg and saved as /content/drive/MyDrive/wikiart/Guided-by-Vector-Flow_line/h.jpg
Processed image da.png and saved as /content/drive/MyDrive/wikiart/Guided-by-Vector-Flow_line/da.png
Processed image f.jpg and saved as /content/drive/MyDrive/wikiart/Guided-by-Vector-Flow_line/f.jpg
Processed image E.png and saved as /content/drive/MyDrive/wikiart/Guided-by-Vector-Flow_line/E.png
Processed image dollar.jpg and saved as /content/drive/MyDrive/wikiart/Guided-by-Vector-Flow_line/dollar.jpg
Processed image dawei.png and saved as /content/drive/MyDrive/wikiart/Guided-by-Vector-Flow_line/dawei.png
Processed image dog.png and saved as /content/drive/MyDrive/wikiart/Guided-by-Vector-Flow_line/dog.png
Processed image field.png and saved as /content/drive/MyDrive/wikiart/Guided-by-Vector-Flow_line/field.png
Processed

SVG to CSV

In [ ]:
import os

# Define the folder containing SVG files
svg_folder = "/content/drive/MyDrive/wikiart/Paint_tran_svg"
# Define the output folder for CSV files
output_folder = "/content/drive/MyDrive/wikiart/Paint_tran_csv"
os.makedirs(output_folder, exist_ok=True)


# List SVG files in the folder
svg_files = [f for f in os.listdir(svg_folder) if f.endswith('.svg')]
import xml.etree.ElementTree as ET
import pandas as pd
import re

def extract_numeric_values(s):
    # Use regular expression to extract numeric values
    return re.findall(r'[-+]?\d*\.\d+|\d+', s)

def parse_svg(svg_file):
    tree = ET.parse(svg_file)
    root = tree.getroot()
    view_box = root.get('viewBox')
    view_box_values = view_box.split()
    svg_width = float(view_box_values[2])  # width is the third value
    svg_height = float(view_box_values[3])  # height is the fourth value

    paths = []

    # Iterate through path elements
    for path_elem in root.findall('.//{http://www.w3.org/2000/svg}path'):
        path_data = path_elem.get('d')
        fill = path_elem.get('fill')

        # Check if fill color is not "#ffffff"
        if fill != "#ffffff":
            # Extract numeric values from path_data using the custom function
            numeric_values = extract_numeric_values(path_data)

            # Extract the second and third elements
            M_x = numeric_values[0] if len(numeric_values) > 1 else None
            M_y = numeric_values[1] if len(numeric_values) > 2 else None

            # Calculate the length of the path data
            path_length = len(path_data)

            paths.append({
                'path_data': path_data,
                'fill': fill,
                'X_value': M_x,
                'Y_value': M_y,
                'path_length': path_length  # Add path length to the dictionary
            })

    # Create DataFrame from paths list
    df = pd.DataFrame(paths)

    # Add width and height to the DataFrame
    df['width'] = svg_width
    df['height'] = svg_height

    # Sort DataFrame based on 'path_length' in descending order
    df.sort_values(by='path_length', ascending=False, inplace=True)

    return df
def save_to_csv(dataframe, csv_filename):
    dataframe.to_csv(csv_filename, index=False)
# Process each SVG file
for svg_file in svg_files:
    # Parse SVG paths and create a DataFrame
    df = parse_svg(os.path.join(svg_folder, svg_file))

    # Split DataFrame based on path_length condition
    subset_path_length_head = df.iloc[0:3]
    subset_path_length_rest = df.iloc[3:]

    # Define output filenames
    filename_head = os.path.splitext(svg_file)[0] + "_head.csv"
    filename_rest = os.path.splitext(svg_file)[0] + "_rest.csv"

    # Define output paths
    output_path_head = os.path.join(output_folder, filename_head)
    output_path_rest = os.path.join(output_folder, filename_rest)

    # Save CSV files to the output folder
    save_to_csv(subset_path_length_head, output_path_head)
    save_to_csv(subset_path_length_rest, output_path_rest)

    print(f"Processed SVG file: {svg_file}. Saved as: {filename_head} and {filename_rest} in {output_folder}")


Processed SVG file: 3.svg. Saved as: 3_head.csv and 3_rest.csv in /content/drive/MyDrive/wikiart/Paint_tran_csv
Processed SVG file: 1.svg. Saved as: 1_head.csv and 1_rest.csv in /content/drive/MyDrive/wikiart/Paint_tran_csv
Processed SVG file: 2.svg. Saved as: 2_head.csv and 2_rest.csv in /content/drive/MyDrive/wikiart/Paint_tran_csv


In [ ]:
import xml.etree.ElementTree as ET
import pandas as pd
import re
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from scipy.cluster.hierarchy import linkage, fcluster
from scipy.spatial.distance import pdist
import cairosvg

common_path = '/content/drive/MyDrive/wikiart/Paint_tran_csv'
File_name = '3'

def extract_numeric_values(s):
    # Use regular expression to extract numeric values
    return re.findall(r'[-+]?\d*\.\d+|\d+', s)

def hierarchical_clustering(csv_file_path):
    df = pd.read_csv(csv_file_path)
    distances = pdist(df[['X_value', 'Y_value']])
    max_distance = df['width'].iloc[0]/32
    print(max_distance)
    linkage_matrix = linkage(distances, method='ward')
    clusters = fcluster(linkage_matrix, t=max_distance, criterion='distance')
    df['Cluster'] = clusters
    df_sorted = df.sort_values(by='Cluster')
    num_clusters = len(set(clusters))
    print("Number of clusters:", num_clusters)
    return df_sorted


def save_svg_and_png_files_head(df_sorted, output_dir_svg, output_dir_png):
    os.makedirs(output_dir_svg, exist_ok=True)
    os.makedirs(output_dir_png, exist_ok=True)
    width = df_sorted['width'].iloc[0]
    height = df_sorted['height'].iloc[0]
    cumulative_svg_content = f'<svg xmlns="http://www.w3.org/2000/svg" width="{width}" height="{height}">'
    #cumulative_svg_content += f'<rect width="{width}" height="{height}" fill="white"/>\n'
    grouped_clusters = df_sorted.groupby('Cluster')

    for cluster, group in grouped_clusters:
        paths_data = group['path_data'].tolist()
        stroke = group['fill'].tolist()
        #print(stroke)

        svg_content = ''

        for path_data, color in zip(paths_data, stroke):
            svg_content += f'<path d="{path_data}" fill="{color}" />\n'
            #svg_content += f'<path d="{path_data}" stroke="{color}" fill="{"none"}" stroke-width="{"1.00"}" vector-effect="{"non-scaling-stroke"}"/>\n'

        cumulative_svg_content += svg_content

        svg_file_path_cluster = os.path.join(output_dir_svg, f'{File_name}_accumulated_clusters_1_{cluster}.svg')
        with open(svg_file_path_cluster, 'w') as f:
            f.write(cumulative_svg_content + '</svg>')

        png_file_path = os.path.join(output_dir_png, f'{File_name}_accumulated_clusters_1_{cluster}.png')
        cairosvg.svg2png(url=svg_file_path_cluster, write_to=png_file_path)

    return cumulative_svg_content

def save_svg_and_png_files_rest(df_sorted, output_dir_svg, output_dir_png):
    os.makedirs(output_dir_svg, exist_ok=True)
    os.makedirs(output_dir_png, exist_ok=True)
    width = df_sorted['width'].iloc[0]
    height = df_sorted['height'].iloc[0]
    #cumulative_svg_content = f'<svg xmlns="http://www.w3.org/2000/svg" width="{width}" height="{height}">'
    cumulative_svg_content = p_data
    grouped_clusters = df_sorted.groupby('Cluster')

    for cluster, group in grouped_clusters:
        paths_data = group['path_data'].tolist()
        fill = group['fill'].tolist()


        svg_content = ''
        for path_data, color in zip(paths_data, fill):
            svg_content += f'<path d="{path_data}" fill="{color}" />\n'
            #svg_content += f'<path d="{path_data}" stroke="{color}" fill="{"none"}" stroke-width="{"1.00"}" vector-effect="{"non-scaling-stroke"}"/>\n'

        # for path_data, style in zip(paths_data, stroke):
        #     svg_content += f'<path d="{path_data}" style="{stroke}" />\n'
            #svg_content += f'<path d="{path_data}" style="fill: none; stroke: black;stroke-width: 2px;" />\n'

        cumulative_svg_content += svg_content
        f_content = cumulative_svg_content + '</svg>'

        svg_file_path = os.path.join(output_dir_svg, f'{File_name}_accumulated_clusters_2_{cluster}.svg')
        with open(svg_file_path, 'w') as f:
            f.write(f_content)

        png_file_path = os.path.join(output_dir_png, f'{File_name}_accumulated_clusters_2_{cluster}.png')
        cairosvg.svg2png(url=svg_file_path, write_to=png_file_path)

csv_file_path_head = os.path.join(common_path, File_name + '_head.csv')
csv_file_path_rest = os.path.join(common_path, File_name + '_rest.csv')
svg_output_dir = os.path.join(common_path, File_name + 'svg_files')
png_output_dir = os.path.join(common_path, File_name +'png_files')

df_sorted_head = hierarchical_clustering(csv_file_path_head)
df_sorted_rest = hierarchical_clustering(csv_file_path_rest)
p_data = save_svg_and_png_files_head(df_sorted_head, svg_output_dir, png_output_dir)
p_data = save_svg_and_png_files_rest(df_sorted_rest, svg_output_dir, png_output_dir)



16.0
Number of clusters: 3
16.0
Number of clusters: 142
